Import all libararies and load the usefull data from the webpage

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np

# get the table's original data
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

# get all data of the table and make a list
data = []
table = soup.find('table', class_='wikitable sortable')

for raw_data  in  table.find_all('td'):
    all_data=raw_data.text
    all_data=all_data.rstrip()
    data.append(all_data)

# creat the dataframe
data_array=np.array(data)
df = pd.DataFrame(data_array.reshape(-1,3), columns = {"PostalCode","Borough","Neighborhood"})
df.columns=['PostalCode','Borough','Neighborhood']
df.head()
df.shape

(289, 3)

Cleaning dataframe 

In [5]:
# get rid of the rows that Borough is 'Not assigned'
df_clean=df[df.Borough != 'Not assigned']

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df.Neighborhood = df.Borough.where(df.Neighborhood == 'Not assigned', df.Neighborhood)

#combine multiple rows with the same Postcode into one row with the neighborhoods separated with a comma 
df_final=df_clean.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()

df_final.head(12)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [3]:
df_final.shape

(103, 3)

Save the dataframe as a csv file

In [4]:
df_final.to_csv("/home/leo/Downloads/Neighborhood.csv",index=False)